In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [2]:
df_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
num = df_train._get_numeric_data()

num[num < 0] = 0

df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,0.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
df_train['date'] = pd.to_datetime(df_train['date'])

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype         
---  ------          -----         
 0   date            datetime64[ns]
 1   date_block_num  int64         
 2   shop_id         int64         
 3   item_id         int64         
 4   item_price      float64       
 5   item_cnt_day    float64       
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 134.4 MB


In [6]:
import plotly.express as px
item_cnt_month = df_train.groupby(['date_block_num'])['item_cnt_day'].sum()

fig = px.bar(item_cnt_month, title="Items Counted per Month", labels={"date_block_num":"Month", "value":"Count"})
fig.show()

In [7]:
# the items

df_items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [8]:
# Reading in item_categories

df_item_cat = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
df_item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [9]:
# Extracting sub categories from item_category_name

## Spliting the category by '-'
df_item_cat['categories'] = df_item_cat["item_category_name"].str.split('-')

## Extracting the first element from split
df_item_cat['type'] = df_item_cat['categories'].apply(lambda x: x[0].strip())

## Extracting second element if there is a second element, else return first element 
df_item_cat['sub_type'] = df_item_cat['categories'].apply(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())

df_item_cat.drop(['item_category_name', 'categories'], inplace=True, axis=1)

df_item_cat.head()

,item_category_id,type,sub_type
0,0,PC,Гарнитуры/Наушники
1,1,Аксессуары,PS2
2,2,Аксессуары,PS3
3,3,Аксессуары,PS4
4,4,Аксессуары,PSP


In [10]:

# the shops

df_shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [11]:
print("Number of unique shops: {}".format(len(df_train["shop_id"].unique())))

Number of unique shops: 60


In [12]:
# Replacing shop ids with other duplicate shop_ids

## Shopnames for 0 and 57 are the same so changing shop_id 0 to 57
df_shops.loc[df_shops['shop_id']==0, 'shop_id'] = 57

## Shopnames for 1 and 58 are the same so changing shop_id 1 to 58
df_shops.loc[df_shops['shop_id']==1, 'shop_id'] = 58

## Shopnames for 10 and 11 are the same so changing shop_id 00 to 11
df_shops.loc[df_shops['shop_id']==10, 'shop_id'] = 11

df_shops = df_shops.drop_duplicates(subset='shop_id')

df_shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",57
1,"!Якутск ТЦ ""Центральный"" фран",58
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [13]:
# Picking up the city names from shop_name

df_shops['city'] = df_shops['shop_name'].str.split(' ').apply(lambda x: x[0])

df_shops.head()

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",57,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",58,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [14]:
df_shops.loc[df_shops['city'] == '!Якутск', 'city'] = 'Якутск'
df_shops

,shop_name,shop_id,city
0,"!Якутск Орджоникидзе, 56 фран",57,Якутск
1,"!Якутск ТЦ ""Центральный"" фран",58,Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский
5,"Вологда ТРЦ ""Мармелад""",5,Вологда
6,"Воронеж (Плехановская, 13)",6,Воронеж
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж
9,Выездная Торговля,9,Выездная


In [15]:
print("Number of unique shops: {}".format(len(df_shops['shop_id'].unique())))

Number of unique shops: 57


In [16]:
# Replacing shop ids in sales_train

## Shopnames for 0 and 57 are the same so changing shop_id 0 to 57
df_train.loc[df_train['shop_id']==0, 'shop_id'] = 57

## Shopnames for 1 and 58 are the same so changing shop_id 1 to 58
df_train.loc[df_train['shop_id']==1, 'shop_id'] = 58

## Shopnames for 10 and 11 are the same so changing shop_id 00 to 11
df_train.loc[df_train['shop_id']==10, 'shop_id'] = 11

In [17]:
#Merging all the dataframes

#Merging df_train with df_item
df_merg = pd.merge(df_train, df_items, how="left", on="item_id")

#Merging df_merg and df_item_cat
df_merg = pd.merge(df_merg, df_item_cat, how="left", on="item_category_id")

#Merging df_merge and df_shops
df_merg = pd.merge(df_merg, df_shops, how="left", on="shop_id")

df_merg.shape

(2935849, 12)

In [18]:
#Grouping data by month, shop_id, item_id to get total sales

df = df_merg.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_price':'mean', 'item_cnt_day':'sum'}).reset_index()
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,2,27,2499.0,1.0
1,0,2,33,499.0,1.0
2,0,2,317,299.0,1.0
3,0,2,438,299.0,1.0
4,0,2,471,399.0,2.0


In [19]:
#Merging df_train with df_item
df = pd.merge(df, df_items, how="left", on="item_id")

#Merging df_merg and df_item_cat
df = pd.merge(df, df_item_cat, how="left", on="item_category_id")

#Merging df_merg and df_shops
df = pd.merge(df, df_shops, how="left", on="shop_id")

df.shape

(1609124, 11)

In [20]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,type,sub_type,shop_name,city
0,0,2,27,2499.0,1.0,"007 Legends [PS3, русская версия]",19,Игры,PS3,"Адыгея ТЦ ""Мега""",Адыгея
1,0,2,33,499.0,1.0,1+1 (BD),37,Кино,Blu,"Адыгея ТЦ ""Мега""",Адыгея
2,0,2,317,299.0,1.0,1С:Аудиокниги. Мединский В. Мифы о России. О р...,45,Книги,Аудиокниги 1С,"Адыгея ТЦ ""Мега""",Адыгея
3,0,2,438,299.0,1.0,1С:Аудиотеатр. Лучшие произведения русских пис...,45,Книги,Аудиокниги 1С,"Адыгея ТЦ ""Мега""",Адыгея
4,0,2,471,399.0,2.0,1С:Бухгалтерия 8 (ред.3.0) как на ладони. Изд ...,49,Книги,Методические материалы 1С,"Адыгея ТЦ ""Мега""",Адыгея


In [21]:
#to sync the month and data_block_num columns 
df['month'] = df['date_block_num'].apply(lambda month: (month+1)%12)

#get_dummies operations
df = pd.concat([df, pd.get_dummies(df['shop_id'], drop_first=True, prefix='shop_')], axis=1)
df = pd.concat([df, pd.get_dummies(df['type'], drop_first=True, prefix='type')], axis=1)
df = pd.concat([df, pd.get_dummies(df['sub_type'], drop_first=True, prefix='sub_type')], axis=1)

df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,type,sub_type,shop_name,...,sub_type_Стандартные издания,sub_type_Сувениры,sub_type_Сувениры (в навеску),"sub_type_Сумки, Альбомы, Коврики д/мыши",sub_type_Фигурки,sub_type_Художественная литература,sub_type_Цифра,sub_type_Чистые носители (шпиль),sub_type_Чистые носители (штучные),sub_type_Элементы питания
0,0,2,27,2499.0,1.0,"007 Legends [PS3, русская версия]",19,Игры,PS3,"Адыгея ТЦ ""Мега""",...,0,0,0,0,0,0,0,0,0,0
1,0,2,33,499.0,1.0,1+1 (BD),37,Кино,Blu,"Адыгея ТЦ ""Мега""",...,0,0,0,0,0,0,0,0,0,0
2,0,2,317,299.0,1.0,1С:Аудиокниги. Мединский В. Мифы о России. О р...,45,Книги,Аудиокниги 1С,"Адыгея ТЦ ""Мега""",...,0,0,0,0,0,0,0,0,0,0
3,0,2,438,299.0,1.0,1С:Аудиотеатр. Лучшие произведения русских пис...,45,Книги,Аудиокниги 1С,"Адыгея ТЦ ""Мега""",...,0,0,0,0,0,0,0,0,0,0
4,0,2,471,399.0,2.0,1С:Бухгалтерия 8 (ред.3.0) как на ладони. Изд ...,49,Книги,Методические материалы 1С,"Адыгея ТЦ ""Мега""",...,0,0,0,0,0,0,0,0,0,0


In [22]:
df.shape

(1609124, 151)

In [23]:
#shop features
shop_cols = [col for col in df.columns if 'shop__' in col]

#type feature names
type_cols = [col for col in df.columns if 'type_' in col]

#sub_type feature names
sub_type_cols = [col for col in df.columns if 'sub_type_' in col]

features = ['month', 'shop_id','item_id', 'item_price'] + type_cols + sub_type_cols
target = ['item_cnt_day']

In [24]:
#the feature and target variables

features = ['month', 'shop_id','item_id', 'item_price'] + type_cols + sub_type_cols
target = ['item_cnt_day']

In [25]:
#modeling
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

from sklearn.model_selection import train_test_split

#splitting data

X = df[features].fillna(value=0)
y = df[target].fillna(value=0)

#splitting and fitting train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr.fit(X_train, y_train)

LinearRegression()

In [26]:
lr.score(X_test, y_test)

0.26585817923119226

In [27]:
#prediction starts

df_test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [28]:
df_test.shape

(214200, 3)

In [29]:
df_test['date_block_num'] = 34
df_test['month'] = 11

item_price = df[['item_id', 'item_price']].groupby('item_id')['item_price'].mean().reset_index()

In [30]:
#Merging df_train with item
df_test = pd.merge(df_test, item_price, how="left", on="item_id")

#Merging df_items with test
df_test = pd.merge(df_test, df_items, how="left", on="item_id")

#Merging df and df_item_cat
df_test = pd.merge(df_test, df_item_cat, how="left", on="item_category_id")

#Merging df and shops
df_test = pd.merge(df_test, df_shops, how="left", on="shop_id")

df_test.shape

(214200, 12)

In [31]:
#dummies and concatenates
df_test = pd.concat([df_test, pd.get_dummies(df_test['shop_id'], drop_first=True, prefix='shop_')], axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_test['type'], drop_first=True, prefix='type')], axis=1)
df_test = pd.concat([df_test, pd.get_dummies(df_test['sub_type'], drop_first=True, prefix='sub_type')], axis=1)

#shop features 
test_shop_cols = [col for col in df_test.columns if 'shop__' in col]
#type feature names
test_type_cols = [col for col in df_test.columns if 'type_' in col]

#sub_type feature names
test_sub_type_cols = [col for col in df_test.columns if 'sub_type_' in col]

test_features = ['month', 'item_id' + 'shop_id', 'item_price'] + test_type_cols + test_sub_type_cols

In [32]:
print(f"Number of predictors in train data: {len(features)}")
print(f"Number of predictors in test data: {len(test_features)}")

Number of predictors in train data: 151
Number of predictors in test data: 114


In [33]:
common_features = list(set(features) & set(test_features)) 
print(f"Number of common features: {len(common_features)}")

Number of common features: 65


In [34]:
'item_price' in common_features

True

In [35]:
#modelling
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
from sklearn.model_selection import train_test_split

#feature and target 
X = df[common_features].fillna(value=0)
y = df[target].fillna(value=0)

#train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

lr.fit(X_train, y_train)


LinearRegression()

In [36]:
lr.score(X_test, y_test)

0.26064816396474855

In [37]:
'item_price' in df_test.columns

True

In [38]:
df_test = df_test.fillna(0)

In [39]:
lr.predict(df_test[common_features])

array([[2.65840172],
       [1.27691155],
       [2.41209199],
       ...,
       [1.3206834 ],
       [1.79007487],
       [1.44407588]])

In [40]:
df_test['preds'] = lr.predict(df_test[common_features])

In [41]:
df_test['preds']

0         2.658402
1         1.276912
2         2.412092
3         2.224542
4         3.863831
            ...   
214195    1.316791
214196    1.591967
214197    1.320683
214198    1.790075
214199    1.444076
Name: preds, Length: 214200, dtype: float64

In [42]:
df_test.head()

,ID,shop_id,item_id,date_block_num,month,item_price,item_name,item_category_id,type,sub_type,...,sub_type_Развитие,sub_type_Служебные,sub_type_Стандартные издания,sub_type_Сувениры,sub_type_Сувениры (в навеску),"sub_type_Сумки, Альбомы, Коврики д/мыши",sub_type_Фигурки,sub_type_Цифра,sub_type_Элементы питания,preds
0,0,5,5037,34,11,1960.580473,"NHL 15 [PS3, русские субтитры]",19,Игры,PS3,...,0,0,0,0,0,0,0,0,0,2.658402
1,1,5,5320,34,11,0.000000,ONE DIRECTION Made In The A.M.,55,Музыка,CD локального производства,...,0,0,0,0,0,0,0,0,0,1.276912
2,2,5,5233,34,11,844.516003,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры,PS3,...,0,0,0,0,0,0,0,0,0,2.412092
3,3,5,5232,34,11,792.527697,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры,XBOX 360,...,0,0,0,0,0,0,0,0,0,2.224542
4,4,5,5268,34,11,0.000000,"Need for Speed [PS4, русская версия]",20,Игры,PS4,...,0,0,0,0,0,0,0,0,0,3.863831


In [43]:
preds = df_test[['ID', 'preds']]

In [44]:
preds.columns = ['ID', 'item_cnt_month']

In [45]:
preds

,ID,item_cnt_month
0,0,2.658402
1,1,1.276912
2,2,2.412092
3,3,2.224542
4,4,3.863831
...,...,...
214195,214195,1.316791
214196,214196,1.591967
214197,214197,1.320683
214198,214198,1.790075


In [46]:
preds.to_csv('btm_submission.csv', index=False)